# Понятие об оптимизации

## Практическое задание

В проектируемой локальной сети есть заданное количество компьютеров. <br>
Есть расстояния от каждого компьютера до сервера и до остальных компьютеров. <br>
Маршрутизаторы соединяются с сервером через оптоволокно стоимостью 12 р/м, а компьютеры с маршрутизаторами соединяются через витую пару стоимостью 30 р/м. Каждый маршрутизатор имеет 8 портов для витой пары и один оптический порт.<br>
Сервер имеет неограниченное число оптических портов и портов для витой пары. Компьютер может соединяться с сервером напрямую или через маршрутизатор.<br>
Построить ЦФ для расчёта стоимости оборудования ЛВС в общем виде. Учесть стоимость маршрутизатора, обоснованность его установки.

Входные данные (задаются фиксированными):
    1) N - заданное кол-во компьютеров
    2) sc[N] - массив длиной N, расстояния от сервера до каждого компьютера
    3) сс[N][N] - 2-мерный массив матрица расстояний между компьютерам 
    4) SM - стоимось маршрутизатора

Параметры регулирования, которые надо оптимизировать:
    1) M - кол-во маршрутизаторов
    2) sm[M] - расстояние от сервера до маршрутизаторов, длины оптоволокна 
    3) mc[N] - расстояние от маршрутизаторов до компьютеров, длины витой пары
    4) sum(sm[M]) - общая дляна оптоволокна
    5) sum(mc[N]) - общая длина витой пары
    6) x - кол-во компьютеров, подключенных к серверу напрямую через витую пару без маршрутизатора

В общем случае ЦФ: cost = 12 * sum по всем M(sm) + 30 * (sum по всем N-x(mc) + sum по всем x(sc))  + M * SM, где sm - расстояние от сервера до маршрутизатора, M - кол-во маршрутизаторов, N - кол-во клиентов - рабочих станций, sc - расстояние от сервера до клиента, SM - стоимость маршрутизатора (хаба)   

cost = sum по всем N(min(30 * sc, 12 * sm + 30 * mc, 12 * (sm - 0.5) + 30 * 0.5 + SM))
1) 30 * sc - подключение компьютера к серверу напрямую через витую пару
2) 12 * sm + 30 * mc - подключение через уже существующий маршрутизатор
3) 12 * (sm - 0.5) + 30 * 0.5 + SM - постановка нового маршрутизатора рядом с компом (в 0.5 метре от компьютера)

Программа расчета минимума целевой ф-ции:

In [5]:
N = 5 #заданное кол-во компьютеров 
sc = [10,20,30,40,50]#- массив длиной N, расстояния от сервера до каждого компьютера, отсортированный по возрастанию
cc = [[0,10,20,30,40],[10,0,10,20,30],[20,10,0,10,20],[30,20,10,0,10],[40,30,20,10,0]] #- 2-мерный массив матрица расстояний между компьютерам 
SM = 500 #- стоимось маршрутизатора
M = 0 #начальное колиество маршрутизаторов
#sm = []
#mc = []
sum_cost = 0
hubs = []
#внешний цикл по N от 0 до N-1
for n in range(N):
    #1 вариант: рассчитывыем самый простой вариант - подключение через витую пару
    min_cost = 30 * sc[n]
    selected_m = -1
    #2 вариант: пробуем уменьшить стоимость подключаясь через уже существующие маршрутизаторы
    #проход по всем имеющимся маршрутизаторам
    for m in range(M):
        hub = hubs[m]
        #проверка превышения лимита соединений хаба
        if hub['count_connection'] > 8:
            continue
        wife_n = hub['wife_computer'] #номер "жены" маршрутизатора 
        mc = cc[n][wife_n] #расстояние от маршрутизатора до компа для простоты принимается равным расстоянию от жены маршрутизатора 
        #(жена в пол-метре)
        cost = 12 * hub['sm'] + 30 * mc
        if cost < min_cost:
            min_cost = cost
            selected_m = m
     #3 вариант: вводим новый маршрутизатор и "женим" этот маршрутизатор на компьютере 
     # - ставим новый маршрутизатор в 0.5 м от этого компа!          
    cost = 12 * (sc[n] - 0.5) + 0.5 * 30 + SM #нужно учесть стоимость маршрутизатора
    if cost < min_cost:
        min_cost = cost
        #вводим новый маршрутизатор
        hub = {}
        hub['sm'] = sc[n] - 0.5
        hub['wife_computer'] = n
        hub['count_connection'] = 1
        hubs.append(hub)
        M += 1 #увеличиваем кол-во маршрутизаторов
    else:
        if selected_m >= 0:
            hubs[selected_m]['count_connection'] += 1 #увеличиваем кол-во соединений
    sum_cost += min_cost
    
print('Стоимость установки сети:', sum_cost)
print('Количество маршрутизаторов:', M)




Стоимость установки сети: 3377.0
Количество маршрутизаторов: 1
